<a href="https://colab.research.google.com/github/mlaugharn/math178project/blob/master/brain_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install medpy simpleitk

     |████████████████████████████████| 153kB 5.0MB/s 
     |████████████████████████████████| 42.5MB 72kB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp36-cp36m-linux_x86_64.whl size=753436 sha256=1b3a7ac8e767d7b9a642a1ee3e36cc7100b9682839ced43bb9696d94198561cf
  Stored in directory: /root/.cache/pip/wheels/8c/c9/9c/2c6281c7a72b9fb1570862a4f028af7ce38405008354fbf870
Successfully built medpy


In [5]:
!pwd

/content


In [0]:
#!wget http://cupid.whatbox.ca:8089/brats2015.zip

In [0]:
#os.mkdir('/content/gdrive/My Drive/brats 2015/')

In [0]:
#!unzip brats2015.zip -d '/content/gdrive/My Drive/brats 2015/'

In [0]:
!cd '/content/gdrive/My Drive/'

In [0]:
import os
os.chdir('/content/gdrive/My Drive/')

In [4]:
from medpy.io import load
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import cv2

# image_data, image_header = load('D:\\Downloads\\brats 2015\\test\\HGG_LGG\\brats_2013_pat0103_1\\VSD.Brain.XX.O.MR_T1c.54195.mha')
# image_data = image_data.astype('float32')
# image_data = ((image_data - image_data.min()) / (image_data.max() - image_data.min()) * 255).astype('uint8')

#plt.imshow(image_data[:,:,50])

#plt.show()

# marching cubes ##### marching cubes sucks for this bc it only creates the external surface, discarding any internal geometry
# from skimage import measure

# verts, faces, normals, values = measure.marching_cubes_lewiner(image_data, 0, step_size=10)
# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(111, projection='3d')

# mesh = Poly3DCollection(verts[faces])
# mesh.set_edgecolor('k')
# ax.add_collection3d(mesh)

# ax.set_xlim(0, 240)
# ax.set_ylim(0, 240)
# ax.set_zlim(0, 155)

# plt.tight_layout()
# plt.show()

# for i in range(image_data.shape[2]):
#     x,y = np.linspace(0, image_data[:,:,i].shape[0], image_data[:,:,i].shape[0]), np.linspace(0, image_data[:,:,i].shape[1], image_data[:,:,i].shape[1])
#     X, Y = np.meshgrid(x,y)
#     fig = plt.figure()
#     ax = plt.axes(projection='3d')
#     ax.contour3D(X,Y, image_data[:, :, i], levels=20, cmap='binary')
#     plt.show()


'''
maybe should be based on SAU-NET
https://openreview.net/pdf?id=wlszIiXbfS
takes in slices, does some attention stuff on each slice, combines adjacent slices, and returns segmented output

the training files have ground truth labels (same slice layout) attached in .OT mha files
'''

'''
we take in a massive tensor of shape

[[2D image slice_1], [2D image slice_2], ... [2D image slice_n]]

we feed it into a UNet model


ENCODERS                                                                                              DECODERS

[small image convolution filters]                    ->skip connections->             [small image convolution filters]
                |                                                                           ^
                |                                                                           |
                v                                                                           |
         [medium image convolution filters]         ->skip connections->        [medium image convolution filters]
                                            \                                   ^
                                             \                                /
                                              \                              /
                                               `>                          /
                                            [large image convolution filters]

the skip connections are basically channels that allow earlier input data to bypass some training layers to be merged into the inputs for later layers



this unet outputs 3D slices of regions it thinks contains tumors
we should find some way to incorporate manifold learning into how we connect adjacent layers

basic outline of steps

1. load data (perhaps splitting into training/validation set)
2. define model (inputs, layers, outputs)
3. define how model updates in response to cost/error/loss function
4. train model
5. with trained model, make predictions on test set

some info found here: http://academictorrents.com/details/c4f39a0a8e46e8d2174b8a8a81b9887150f44d50

>>> 1 for necrosis
>>> 2 for edema
>>> 3 for non-enhancing tumor
>>> 4 for enhancing tumor
>>> 0 for everything else
'''

import keras
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import *
from keras import backend as K
#K.set_image_data_format('channels_last')
import glob
import os.path
from scipy.ndimage import zoom
import numpy as np
class BrainSegmentationUNet:
    def __init__(self):
        #self.dataset_location = r'D:/Downloads/brats 2015/'
        self.dataset_location = r'/content/gdrive/My Drive/brats 2015/BRATS2015/'
        self.batch_size = 1
        print(glob.glob(self.dataset_location))
        self.dataset_size = len(glob.glob(self.dataset_location + r'training/HGG/**/'))
        print(f"dataset size: {self.dataset_size}")
        for patient_folder in glob.glob(self.dataset_location + r'training/HGG/**/'):
            patient_id = os.path.basename(patient_folder)
            mha_filenames = glob.glob(patient_folder + r'/*.mha')
            Flair_filename, T1_filename, T1c_filename, T2_filename, ground_truth_filename = mha_filenames
            
            Flair_data, Flair_header = load(Flair_filename)
            T1_data, T1_header = load(T1_filename)
            T1c_data, T1c_header = load(T1c_filename)
            T2_data, T2_header = load(T2_filename)
            ground_truth_data, ground_truth_header = load(ground_truth_filename)
            
            self.Flair_shape        = [256, 256, 256] #[ax * 1 for ax in Flair_data.shape]
            self.T1_shape           = [ax * 1 for ax in T1_data.shape]
            self.T1c_shape          = [ax * 1 for ax in T1c_data.shape]
            self.T2_shape           = [ax * 1 for ax in T2_data.shape]
            self.ground_truth_shape = [ax * 1 for ax in ground_truth_data.shape]

            break
        self.setup_model()
    def data_generator(self, train=True):
        if train:
            # have HGG and LGG files

            # HGGs
            # you will find 5 files in HGG folders:
            # eg in D:\Downloads\brats 2015\train\HGG\brats_2013_pat0001_1\
            # you have

            # "D:\Downloads\brats 2015\train\HGG\brats_2013_pat0001_1\VSD.Brain.XX.O.MR_T1.54513.mha"
            # "D:\Downloads\brats 2015\train\HGG\brats_2013_pat0001_1\VSD.Brain.XX.O.MR_T1c.54514.mha"
            # "D:\Downloads\brats 2015\train\HGG\brats_2013_pat0001_1\VSD.Brain.XX.O.MR_T2.54515.mha"
            # "D:\Downloads\brats 2015\train\HGG\brats_2013_pat0001_1\VSD.Brain_3more.XX.O.OT.54517.mha"
            # "D:\Downloads\brats 2015\train\HGG\brats_2013_pat0001_1\VSD.Brain.XX.O.MR_Flair.54512.mha"

            # first 4 are mri scans
            # last one is the ground truth labels
            while True:
                batch_X = []
                batch_y = []

                for patient_folder in glob.glob(self.dataset_location + r'training/HGG/**/'):
                    patient_id = os.path.basename(patient_folder)
                    mha_filenames = glob.glob(patient_folder + r'/*.mha')
                    print(mha_filenames)
                    Flair_filename, T1_filename, T1c_filename, T2_filename, ground_truth_filename = mha_filenames
                    
                    Flair_data, Flair_header = load(Flair_filename)
                    T1_data, T1_header = load(T1_filename)
                    T1c_data, T1c_header = load(T1c_filename)
                    T2_data, T2_header = load(T2_filename)
                    ground_truth_data, ground_truth_header = load(ground_truth_filename)

                    Flair_data = zoom(Flair_data, (256/240,256/240,256/155))
                    # T1_data = zoom(T1_data, (4,4,4))
                    # T1c_data = zoom(T1c_data, (4,4,4))
                    # T2_data = zoom(T2_data, (4,4,4))
                    ground_truth_data = zoom(ground_truth_data, (256/240, 256/240, 256/155))
                    
                    self.Flair_shape        = Flair_data.shape
                    self.T1_shape           = T1_data.shape
                    self.T1c_shape          = T1c_data.shape
                    self.T2_shape           = T2_data.shape
                    self.ground_truth_shape = ground_truth_data.shape

                    #batch_X = [Flair_data, T1_data, T1c_data, T2_data] # batch size = 1
                    batch_X = np.expand_dims(np.expand_dims(Flair_data, axis=0), axis=-1)
                    batch_y = [np.expand_dims(ground_truth_data, axis=-1)]
                    
                    #if len(batch_X) == self.batch_size:
                    #batch_X = [[np.array(sub_X) for sub_X in batch_X]]
                    batch_y = np.array(batch_y)
                    print("batch prepared!...")
                    yield batch_X, batch_y
                    batch_X, batch_y = [], []
    
    def setup_model(self):
        # take in arrays of 2D images
        # feed into UNet
        # use manifold esque attention between slice layers
        # feed to unet decoder 

        input_layers = []
        
        # Flair_unet
        def make_unet(input_layer_shape):
            input_layer = Input(input_layer_shape)
            input_layers.append(input_layer)
            conv1 = Conv3D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(input_layer)
            conv1 = Conv3D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
            pool1 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv1)
            conv2 = Conv3D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
            conv2 = Conv3D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
            pool2 = MaxPooling3D(pool_size=(2, 2, 2), padding = 'same')(conv2)
            conv3 = Conv3D(256, 3, activation='relu', padding='same', kernel_initializer = 'he_normal')(pool2)
            conv3 = Conv3D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
            pool3 = MaxPooling3D(pool_size=(2, 2, 2), padding = 'same')(conv3)
            conv4 = Conv3D(512, 3, activation='relu', padding='same', kernel_initializer = 'he_normal')(pool3)
            conv4 = Conv3D(512, 3, activation='relu', padding='same', kernel_initializer = 'he_normal')(conv4)
            drop4 = Dropout(0.5)(conv4)
            pool4 = MaxPooling3D(pool_size=(2, 2, 2), padding = 'same')(drop4)
            
            conv5 = Conv3D(1024, 3, activation='relu', padding='same', kernel_initializer = 'he_normal')(pool4)
            conv5 = Conv3D(1024, 3, activation='relu', padding='same', kernel_initializer = 'he_normal')(conv5)
            drop5 = Dropout(0.5)(conv5)

            up6 = Conv3D(512, 2, activation='relu', padding='same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(drop5))
            merge6 = concatenate([drop4,up6], axis = 4) # TODO fix this <--
            conv6 = Conv3D(512, 3, activation='relu', padding='same', kernel_initializer = 'he_normal')(merge6)
            conv6 = Conv3D(512, 3, activation='relu', padding='same', kernel_initializer = 'he_normal')(conv6)

            up7 = Conv3D(256, 2, activation='relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv6))
            merge7 = concatenate([conv3, up7], axis=4)
            conv7 = Conv3D(256, 3, activation='relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
            conv7 = Conv3D(256, 3, activation='relu', padding = 'same', kernel_initializer ='he_normal')(conv7)

            up8 = Conv3D(256, 2, activation='relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv7))
            merge8 = concatenate([conv2, up8], axis=4)
            conv8 = Conv3D(128, 3, activation='relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
            conv8 = Conv3D(128, 3, activation='relu', padding = 'same', kernel_initializer ='he_normal')(conv8)

            up9 = Conv3D(64, 2, activation='relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv8))
            merge9 = concatenate([conv1, up9], axis=4)
            conv9 = Conv3D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
            conv9 = Conv3D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

            conv9 = Conv3D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

            conv10 = Conv3D(1,1, activation='sigmoid')(conv9)
            return conv10
        per_mode_unet_outputs = []
        # for input_layer in range(1):#self.Flair_shape[-1]):#, self.T1_shape, self.T1c_shape, self.T2_shape]:
        #     print(input_layer)
        #     conv10 = make_unet((240,240,155))#input_layer)
        #     per_mode_unet_outputs.append(conv10)
        #merged_unet_outputs = concatenate(per_mode_unet_outputs, axis=3)
        merged_unet_outputs = make_unet((256,256,256,1))
        final = Conv3D(1,1, activation = 'sigmoid')(merged_unet_outputs)
        self.model = Model(input = input_layers, output = final)
        self.model.compile(optimizer = Adam(lr= 1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        print(self.model.summary())

    def train(self):
        self.model.fit_generator(self.data_generator(), steps_per_epoch = self.dataset_size, epochs = 1)

    def predict(self):
        pass

#example unet:
# def unet(pretrained_weights = None,input_size = (256,256,1)):
#     inputs = Input(input_size)
#     conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
#     conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
#     conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
#     conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
#     conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
#     conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
#     conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
#     conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
#     drop4 = Dropout(0.5)(conv4)
#     pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

#     conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
#     conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
#     drop5 = Dropout(0.5)(conv5)

#     up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
#     merge6 = concatenate([drop4,up6], axis = 3)
#     conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
#     conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

#     up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
#     merge7 = concatenate([conv3,up7], axis = 3)
#     conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
#     conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

#     up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
#     merge8 = concatenate([conv2,up8], axis = 3)
#     conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
#     conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

#     up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
#     merge9 = concatenate([conv1,up9], axis = 3)
#     conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
#     conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#     conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#     conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

#     model = Model(input = inputs, output = conv10)

# bsu = BrainSegmentationUNet()
# bsu.train()

Using TensorFlow backend.


In [0]:
from medpy.io import load
import cv2
import SimpleITK as sitk
from tqdm.autonotebook import tqdm
import glob
import os
import pathlib
import shutil
def prepare_Flair_MRI_slices():
    dataset_location = r'/content/gdrive/My Drive/brats 2015/BRATS2015/'
    patient_folders = glob.glob(dataset_location + r'training/HGG/**/')
    for folder_num, patient_folder in enumerate(tqdm(patient_folders)):
        #print(f'folder {folder_num}/{len(patient_folders)}:')
        #print(patient_folder)
        patient_id = os.path.basename(patient_folder)
        mha_filenames = glob.glob(patient_folder + r'/*.mha')
        #Flair_filename, T1_filename, T1c_filename, T2_filename, ground_truth_filename = mha_filenames
        Flair_filename = list(filter(lambda fname: 'Flair' in fname, mha_filenames))[0]
        ground_truth_filename = list(filter(lambda fname: '.OT' in fname, mha_filenames))[0]
        
        #Flair_data, Flair_header = load(Flair_filename)
        Flair_data = sitk.ReadImage(Flair_filename)
        Flair_data = sitk.Cast(sitk.RescaleIntensity(Flair_data), sitk.sitkUInt8)

        ground_truth_data, ground_truth_header = load(ground_truth_filename)
        
        # we slice by x, y, and z
        # put em each into a different folder
        if os.path.exists(patient_folder + 'slices/'): shutil.rmtree(patient_folder + 'slices/')
        if not os.path.exists(patient_folder + 'slices/flair/'):
            pathlib.Path(patient_folder + 'slices/flair/').mkdir(parents=True, exist_ok=True)
            # make x y z folders
        for ax in ['x','y','z']: pathlib.Path(patient_folder + f'slices/flair/{ax}/').mkdir(parents=True, exist_ok=True)
        # print('making flair slices')
        # print(Flair_data.GetSize())
        # print('creating x slices')
        for idx in range(240): # iterate x
            slice_img = Flair_data[idx, :, :]
            slice_filename = f'{idx}.png'
            sitk.WriteImage(slice_img, patient_folder + f'slices/flair/x/{slice_filename}')
            #print(patient_folder + f'slices/flair/x/{slice_filename}')
        #print('creating y slices')
        for idx in range(240): # iterate y
            slice_img = Flair_data[:, idx, :]
            slice_filename = f'{idx}.png'
            sitk.WriteImage(slice_img, patient_folder + f'slices/flair/y/{slice_filename}')
            #print(patient_folder + f'slices/flair/y/{slice_filename}')
        #print('creating z slices')
        for idx in range(155): # iterate z
            slice_img = Flair_data[:, :, idx]
            slice_filename = f'{idx}.png'
            sitk.WriteImage(slice_img, patient_folder + f'slices/flair/z/{slice_filename}')
            #print(patient_folder + f'slices/flair/z/{slice_filename}')
        if not os.path.exists(patient_folder + 'slices/truth/'):
            pathlib.Path(patient_folder + 'slices/truth/').mkdir(parents=True, exist_ok=True)
        for ax in ['x','y','z']: pathlib.Path(patient_folder + f'/slices/truth/{ax}').mkdir(parents=True, exist_ok=True)
        #print('making truth slices')
        # make ground truth images
        # these only have the numbers 0-5 at each pixel location so no scaling necessary, and it's easier to just use medpy for this
        # without having to use the (much slower) sitk
        print('creating x slices')
        for idx in range(240): # ground_truth comes from medpy so its a numpy array, x iteration
            slice_img = ground_truth_data[idx, :, :]
            slice_filename = f'{idx}.png'
            cv2.imwrite(patient_folder + f'slices/truth/x/{slice_filename}', slice_img)
            #print(patient_folder + f'slices/truth/x/{slice_filename}')
        #print('creating y slices')
        for idx in range(240): # y iteration
            slice_img = ground_truth_data[:, idx, :]
            slice_filename = f'{idx}.png'
            cv2.imwrite(patient_folder + f'slices/truth/y/{slice_filename}', slice_img)
            #print(patient_folder + f'slices/truth/y/{slice_filename}')
        #print('creating x slices')
        for idx in range(155): # z iteration
            slice_img = ground_truth_data[:, :, idx]
            slice_filename = f'{idx}.png'
            cv2.imwrite(patient_folder + f'slices/truth/z/{slice_filename}', slice_img)
            #print(patient_folder + f'slices/truth/z/{slice_filename}')


In [60]:
prepare_Flair_MRI_slices()

creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x slices
creating x

In [0]:
NUM_TRAIN = 220
BATCH_SIZE_TRAIN = 1 # 
def images_generator(): # lets just try x axis images only for our first attempt

    dataset_location = r'/content/gdrive/My Drive/brats 2015/BRATS2015/'
    print(dataset_location)
    patient_folders = sorted(glob.glob(dataset_location + r'training/HGG/**/'))
    print(patient_folders)
    NUM_TRAIN = len(patient_folders)
    
    batch_size = BATCH_SIZE_TRAIN
    batch_X = []
    batch_y = []
    while True:
        for folder in patient_folders:
            flair_imgs = folder + 'slices/flair/z'
            truth_imgs = folder + 'slices/truth/z'
            truth_shapes = []
            for i, (flair_slice_img, truth_slice_img) in enumerate(zip(glob.glob(flair_imgs + '/*.png'), glob.glob(truth_imgs + '/*.png'))):
                if i == 155: break
                #print(f'reading {flair_slice_img}, {truth_slice_img}')
                flair_img = cv2.imread(flair_slice_img, 0) / 255
                flair_img = np.expand_dims(flair_img, axis=-1)
                batch_X.append(flair_img)

                truth_img = cv2.imread(truth_slice_img, 0)
                truth_img = np.expand_dims(truth_img, axis=-1)
                truth_shapes.append(truth_img.shape)
                batch_y.append(keras.utils.to_categorical(truth_img, num_classes=5))

                if len(batch_X) == batch_size:
                    batch_X = np.array(batch_X)
                    #print(batch_y)
                    #print(set(truth_shapes))
                    batch_y = np.array(batch_y)
                    #print(batch_y.shape)
                    #print('giving batch')
                    yield batch_X, batch_y
                    batch_X = []
                    batch_y = []
            

In [0]:
IMAGE_HEIGHT = 240
IMAGE_WIDTH = 240
def unet(n_levels, initial_features=32, n_blocks=2, kernel_size=3, pooling_size=2, in_channels=1, out_channels=1):
    inputs = keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, in_channels))
    x = inputs
    
    convpars = dict(kernel_size=kernel_size, activation='relu', padding='same')
    
    #downstream
    skips = {}
    for level in range(n_levels):
        for _ in range(n_blocks):
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)
        if level < n_levels - 1:
            skips[level] = x
            x = keras.layers.MaxPool2D(pooling_size)(x)
            
    # upstream
    for level in reversed(range(n_levels-1)):
        x = keras.layers.Conv2DTranspose(initial_features * 2 ** level, strides=pooling_size, **convpars)(x)
        x = keras.layers.Concatenate()([x, skips[level]])
        for _ in range(n_blocks):
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)
            
    # output
    activation = 'sigmoid' if out_channels == 1 else 'softmax'
    x = keras.layers.Conv2D(out_channels, kernel_size=1, activation=activation, padding='same')(x)
    
    return keras.Model(inputs=[inputs], outputs=[x], name=f'UNET-L{n_levels}-F{initial_features}')

In [68]:
import keras
EPOCH_STEP_TRAIN = NUM_TRAIN // BATCH_SIZE_TRAIN
print(EPOCH_STEP_TRAIN)
#EPOCH_STEP_TEST = NUM_TEST // BATCH_SIZE_TEST

model = unet(4, out_channels=5)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

220


In [69]:
model.summary()

Model: "UNET-L4-F32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 240, 240, 1)  0                                            
__________________________________________________________________________________________________
conv2d_151 (Conv2D)             (None, 240, 240, 32) 320         input_11[0][0]                   
__________________________________________________________________________________________________
conv2d_152 (Conv2D)             (None, 240, 240, 32) 9248        conv2d_151[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_31 (MaxPooling2D) (None, 120, 120, 32) 0           conv2d_152[0][0]                 
________________________________________________________________________________________

In [70]:
NUM_OF_EPOCHS = 10
# model.fit_generator(generator=images_generator(), 
#                     steps_per_epoch=EPOCH_STEP_TRAIN, 
#                     # validation_data=test_generator, 
#                     # validation_steps=EPOCH_STEP_TEST,
#                    epochs=NUM_OF_EPOCHS)
model.fit(images_generator(), steps_per_epoch=EPOCH_STEP_TRAIN, epochs=NUM_OF_EPOCHS)

Epoch 1/10
/content/gdrive/My Drive/brats 2015/BRATS2015/
['/content/gdrive/My Drive/brats 2015/BRATS2015/training/HGG/brats_2013_pat0001_1/', '/content/gdrive/My Drive/brats 2015/BRATS2015/training/HGG/brats_2013_pat0002_1/', '/content/gdrive/My Drive/brats 2015/BRATS2015/training/HGG/brats_2013_pat0003_1/', '/content/gdrive/My Drive/brats 2015/BRATS2015/training/HGG/brats_2013_pat0004_1/', '/content/gdrive/My Drive/brats 2015/BRATS2015/training/HGG/brats_2013_pat0005_1/', '/content/gdrive/My Drive/brats 2015/BRATS2015/training/HGG/brats_2013_pat0006_1/', '/content/gdrive/My Drive/brats 2015/BRATS2015/training/HGG/brats_2013_pat0007_1/', '/content/gdrive/My Drive/brats 2015/BRATS2015/training/HGG/brats_2013_pat0008_1/', '/content/gdrive/My Drive/brats 2015/BRATS2015/training/HGG/brats_2013_pat0009_1/', '/content/gdrive/My Drive/brats 2015/BRATS2015/training/HGG/brats_2013_pat0010_1/', '/content/gdrive/My Drive/brats 2015/BRATS2015/training/HGG/brats_2013_pat0011_1/', '/content/gdrive/

In [38]:
!ls 'brats 2015/BRATS2015/training/HGG/brats_2013_pat0001_1/slices/flair/z'

0.png	 124.png  149.png  173.png  198.png  221.png  2.png   54.png  79.png
100.png  125.png  14.png   174.png  199.png  222.png  30.png  55.png  7.png
101.png  126.png  150.png  175.png  19.png   223.png  31.png  56.png  80.png
102.png  127.png  151.png  176.png  1.png    224.png  32.png  57.png  81.png
103.png  128.png  152.png  177.png  200.png  225.png  33.png  58.png  82.png
104.png  129.png  153.png  178.png  201.png  226.png  34.png  59.png  83.png
105.png  12.png   154.png  179.png  202.png  227.png  35.png  5.png   84.png
106.png  130.png  155.png  17.png   203.png  228.png  36.png  60.png  85.png
107.png  131.png  156.png  180.png  204.png  229.png  37.png  61.png  86.png
108.png  132.png  157.png  181.png  205.png  22.png   38.png  62.png  87.png
109.png  133.png  158.png  182.png  206.png  230.png  39.png  63.png  88.png
10.png	 134.png  159.png  183.png  207.png  231.png  3.png   64.png  89.png
110.png  135.png  15.png   184.png  208.png  232.png  40.png  65.png  8.png
111.

In [43]:
cv2.imread('brats 2015/BRATS2015/training/HGG/brats_2013_pat0001_1/slices/flair/z/200.png', 0).shape

(155, 240)

In [0]:
!cd './brats 2015'

In [0]:
os.chdir('./brats 2015')

In [0]:
!ls

In [0]:
ls BRATS2015/training/HGG/brats/brats_2013_pat0001_1